<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Advanced-Convolutional-Neural-Networks" data-toc-modified-id="Advanced-Convolutional-Neural-Networks-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Advanced Convolutional Neural Networks</a></span></li><li><span><a href="#Classification-and-localization" data-toc-modified-id="Classification-and-localization-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Classification and localization</a></span></li><li><span><a href="#Semantic-segmentation" data-toc-modified-id="Semantic-segmentation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Semantic segmentation</a></span><ul class="toc-item"><li><span><a href="#Approch-1-and-2" data-toc-modified-id="Approch-1-and-2-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Approch 1 and 2</a></span></li><li><span><a href="#Approch--CNN-encoder-decoder-network" data-toc-modified-id="Approch--CNN-encoder-decoder-network-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Approch  CNN encoder-decoder network</a></span></li></ul></li><li><span><a href="#Object-detection" data-toc-modified-id="Object-detection-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Object detection</a></span><ul class="toc-item"><li><span><a href="#classification-and-localization-networks" data-toc-modified-id="classification-and-localization-networks-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>classification and localization networks</a></span></li><li><span><a href="#Selective-Search---Region-Proposals-CNN-(R-CNN)" data-toc-modified-id="Selective-Search---Region-Proposals-CNN-(R-CNN)-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Selective Search - Region Proposals-CNN (R-CNN)</a></span></li><li><span><a href="#SVM-based-classifier" data-toc-modified-id="SVM-based-classifier-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>SVM-based classifier</a></span></li><li><span><a href="#Fast-R-CNN" data-toc-modified-id="Fast-R-CNN-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Fast R-CNN</a></span></li><li><span><a href="#Faster-R-CNN" data-toc-modified-id="Faster-R-CNN-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Faster R-CNN</a></span></li><li><span><a href="#You-Only-Look-Once-(YOLO)" data-toc-modified-id="You-Only-Look-Once-(YOLO)-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>You Only Look Once (YOLO)</a></span></li></ul></li><li><span><a href="#Instance-segmentation" data-toc-modified-id="Instance-segmentation-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Instance segmentation</a></span></li></ul></div>

# Advanced Convolutional Neural Networks

Different computer vision tasks. Source: Introduction to Artificial Intelligence and Computer
Vision Revolution (https://www.slideshare.net/darian_f/introduction-to-the-artificial-intelligence-andcomputer-vision-revolution)

![](./i/computer_vision_tasks.png)





# Classification and localization

In the classification and localization task not only do you have to report the class of
object found in the image, but also the coordinates of the bounding box where the
object appears in the image. This type of task assumes that there is only one instance
of the object in an image.

*This can be achieved by attaching a "regression head"  in addition to the
"classification head" in a typical classification network *

    - classification head:   
    Recall that in a classification network, the final output of convolution and pooling operations, called the feature map, is fed into a fully connected network that produces a vector of class probabilities. This fully connected network is called the `classification head`, and it is tuned using a categorical loss function (Lc) such as `categorical cross entropy`.

    - regression head:
    a regression head is another fully connected network that takes the feature map and produces a vector (x, y, w, h) representing the top-left x and y coordinates, width and height of the bounding box. 
    It is tuned using a continuous loss function (Lr) such as mean squared error. The entire network is tuned using a linear combination of the two losses, that is:
$$
𝐿 = 𝛼𝐿_C + (1 − 𝛼)𝐿_𝑟
$$

Here $𝛼$ is a hyperparameter and can take a value between 0 and 1. Unless the value
is determined by some domain knowledge about the problem, it can be set to 0.5.

![](./i/localizationandclassification_pattern.png)

*the only difference with respect to a typical CNN classification network is the additional regression head on the top right*


# Semantic segmentation
the aim is to classify every single pixel on the image as belonging to a single class.

## Approch 1 and 2

An initial method of implementation could be to build a classifier network for each
pixel, where the input is a small neighborhood around each pixel. In practice, this
approach is not very performant, so an improvement over this implementation
might be to run the image through convolutions that will increase the feature depth,
while keeping the image width and height constant. Each pixel then has a feature
map that can be sent through a fully connected network that predicts the class of the
pixel. However, in practice, this is also quite expensive, and it is not normally used.

## Approch  CNN encoder-decoder network
the `encoder` decreases the width and height of the image but increases its depth (number of
features), while the `decoder` uses transposed convolution operations to increase its
size and decrease depth. Transpose convolution (or upsampling) is the process of
going in the opposite direction of a normal convolution. The input to this network is
the image and the output is the segmentation map.

A popular implementation of this encoder-decoder architecture is the U-Net. (https://github.com/jakeret/tf_unet)

U-Net: Convolutional Networks for Biomedical Image Segmentation:
![](./i/u-net-architecture.png)

# Object detection
The object detection task is similar to the classification and localization tasks. The big
difference is that now there are multiple objects in the image, and for each one we
need to find the class and bounding box coordinates. In addition, neither the number
of objects nor their size is known in advance. As you can imagine, this is a difficult
problem and a fair amount of research has gone into it.


##  classification and localization networks
A first approach to the problem might be to create many random crops of the
input image and for each crop, apply the classification and localization networks
we described earlier. However, such an approach is very wasteful in terms of
computing and unlikely to be very successful.

## Selective Search - Region Proposals-CNN (R-CNN)
A more practical approach would be use a tool such as Selective Search (Selective
Search for Object Recognition, by Uijlings et al.
(http://www.huppelen.nl/publications/selectiveSearchDraft.pdf)

uses traditional computer
vision techniques to find areas in the image that might contain objects. These regions
are called "Region Proposals," and the network to detect them was called "Region
Proposal Network," or R-CNN. In the original R-CNN, the regions were resized
and fed into a network to yield image vectors:

![](./i/R_cnn_cnn.png)

## SVM-based classifier
These vectors were then classified with an `SVM-based classifier` (https://
en.wikipedia.org/wiki/Support-vector_machine), and the bounding boxes
proposed by the external tool were corrected using a linear regression network over
the image vectors. A R-CNN network can be represented conceptually as shown in
Figure 5:

![](./i/R_cnn_Network.png)

## Fast R-CNN
The next iteration of the R-CNN network was called the `Fast R-CNN`. The Fast
R-CNN still gets its region proposals from an external tool, but instead of feeding
each region proposal through the CNN, the entire image is fed through the CNN
and the region proposals are projected onto the resulting feature map. Each region of
interest is fed through an **Region of Interest (ROI)** pooling layer and then to a fully
connected network, which produces a feature vector for the ROI. 

ROI pooling is a widely used operation in object detection tasks using convolutional
neural networks. The ROI pooling layer uses max pooling to convert the features
inside any valid region of interest into a small feature map with a fixed spatial extent
of H × W (where H and W are two hyperparameters). The feature vector is then
fed into two fully connected networks, one to predict the class of the ROI and the
other to correct the bounding box coordinates for the proposal. This is illustrated
in Figure 6:

![](./i/fast_Rnn.png)

## Faster R-CNN
The Fast R-CNN is about 25x faster than the R-CNN. The next improvement, called
the Faster R-CNN (an implementation can be found at https://github.com/tensorpack/tensorpack/tree/master/examples/FasterRCNN), removes the
external region proposal mechanism and replaces it with a trainable component,
called the Region Proposal Network (RPN), within the network itself.

The output of this network is combined with the feature map and passed in
through a similar pipeline to the Fast R-CNN network, as shown in Figure 7. The
Faster R-CNN network is about 10x faster than the Fast R-CNN network, making
it approximately 250x faster than an R-CNN network:

![](./i/faster_Rnn.png)



## You Only Look Once (YOLO)
Another somewhat different class of object detection networks are Single Shot
Detectors (SSD) such as You Only Look Once (YOLO). In these cases, each image is
split into a predefined number of parts using a grid. In the case of YOLO, a 7×7 grid
is used, resulting in 49 subimages.A predetermined set of crops with different aspect
ratios are applied to each sub-image. 

Given B bounding boxes and C object classes,
the output for each image is a vector of size **(7 * 7 * (5B + C))**. Each bounding box has
a confidence and coordinates (x, y, w, h), and each grid has prediction probabilities
for the different objects detected within them.

The YOLO network is a CNN that does this transformation. The final predictions
and bounding boxes are found by aggregating the findings from this vector. In
YOLO a single convolutional network predicts the bounding boxes and the related
class probabilities. YOLO is the faster solution for object detection, but the algorithm
might fail to detect smaller objects (an implementation can be found at 
https://www.kaggle.com/aruchomu/yolo-v3-object-detection-in-tensorflow).


# Instance segmentation

Instance segmentation is similar to semantic segmentation – the process of
associating each pixel of an image with a class label – with a few important
distinctions. 

    - First, it needs to distinguish between different instances of the sameclass in an image. 
    - Second, it is not required to label every single pixel in the image. In some respects, instance segmentation is also similar to object detection, except that instead of bounding boxes, we want to find a binary mask that covers each object.
    
![](./i/mask_rcnn.png)